In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
main_dir = 'D:/Competitions/datastorm-2021'


In [3]:
data_dir = main_dir+'/data/feature/'
train_file = data_dir+'cb_train.csv'
val_file = data_dir+'cb_validation.csv'
test_file = data_dir+'cb_test.csv'

In [4]:
train_ds = pd.read_csv(train_file)
val_ds = pd.read_csv(val_file)


In [5]:
def start_category_from_zero(df,columns):
    for each in columns:
        df[each] = df[each] - 1

    return df

In [6]:
start_category_from_zero(train_ds,['Reservation_Status'])
start_category_from_zero(val_ds,['Reservation_Status'])

,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,...,Booking_channel,Required_Car_Parking,Use_Promotion,Room_Rate,month_Expected_checkin,dayofweek_Expected_checkin,stay,booking_to_checkin,total_participants,Reservation_Status
0,1,56,1,0,0,3,2,2,0,0,...,1,1,0,192,8,2,2,195,4,2
1,1,60,0,2,3,3,2,1,1,0,...,0,0,1,187,8,2,4,175,4,1
2,0,58,3,2,0,0,1,1,0,0,...,2,1,0,227,9,3,1,193,4,1
3,0,23,0,2,3,1,1,1,1,0,...,2,1,0,189,8,2,2,103,3,0
4,0,47,3,2,3,2,0,2,1,0,...,0,0,0,218,8,2,1,92,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744,0,22,1,2,1,2,2,0,1,0,...,0,1,0,199,11,4,3,37,7,0
2745,0,35,1,2,3,2,1,1,0,0,...,2,0,0,106,12,4,1,154,5,1
2746,1,18,3,0,3,0,2,1,0,0,...,1,1,0,136,11,4,2,30,5,2
2747,1,42,3,2,0,3,0,2,1,1,...,0,0,0,226,11,4,1,170,6,0


In [7]:
print(train_ds['Reservation_Status'].value_counts())
train_ds.dtypes

0    21240
1     4134
2     2125
Name: Reservation_Status, dtype: int64


Gender                        int64
Age                           int64
Ethnicity                     int64
Educational_Level             int64
Income                        int64
Country_region                int64
Hotel_Type                    int64
Meal_Type                     int64
Visted_Previously             int64
Previous_Cancellations        int64
Deposit_type                  int64
Booking_channel               int64
Required_Car_Parking          int64
Use_Promotion                 int64
Room_Rate                     int64
month_Expected_checkin        int64
dayofweek_Expected_checkin    int64
stay                          int64
booking_to_checkin            int64
total_participants            int64
Reservation_Status            int64
dtype: object

In [8]:
x_columns = [x for x in train_ds.columns if x != 'Reservation_Status']
y_columns = ['Reservation_Status']
optimizer = tf.keras.optimizers.RMSprop()
batch_size = 128
input_shape = (len(x_columns))
epochs = 200
print(input_shape)

20


In [9]:
x_columns

['Gender',
 'Age',
 'Ethnicity',
 'Educational_Level',
 'Income',
 'Country_region',
 'Hotel_Type',
 'Meal_Type',
 'Visted_Previously',
 'Previous_Cancellations',
 'Deposit_type',
 'Booking_channel',
 'Required_Car_Parking',
 'Use_Promotion',
 'Room_Rate',
 'month_Expected_checkin',
 'dayofweek_Expected_checkin',
 'stay',
 'booking_to_checkin',
 'total_participants']

In [10]:
x_train = np.asarray(train_ds[x_columns])
y_train = np.asarray(train_ds[y_columns])
x_val = np.asarray(val_ds[x_columns])
y_val = np.asarray(val_ds[y_columns])

In [11]:
x_train.shape

(27499, 20)

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(3,activation='softmax'))


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 20)                80        
_________________________________________________________________
dense (Dense)                (None, 32)                672       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32)                1

In [14]:
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer= optimizer,
    metrics=['accuracy'])

In [15]:
history = model.fit(x_train,y_train ,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val,y_val))

Epoch 1/200
215/215 [==============================] - 2s 3ms/step - loss: 1.1515 - accuracy: 0.4707 - val_loss: 1.0053 - val_accuracy: 0.5857
Epoch 2/200
215/215 [==============================] - 0s 1ms/step - loss: 0.7340 - accuracy: 0.7565 - val_loss: 1.0415 - val_accuracy: 0.5857